In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [2]:
import re
import os

In [3]:
from sklearn.metrics import mean_absolute_error
import pickle

# 1. Data Loading & Preparation

In [4]:
df_NTC = pd.read_csv("../../data/NTC.csv")
df_losses = pd.read_csv("../../data/Avtice-losses.csv")

df_renewable = pd.read_csv("../../data/Forecast-renewable-generation.csv")
df_temp = pd.read_csv("../../data/Forecast-temperature.csv")

df_losses = df_losses.drop(df_losses.index[0])

In [5]:
df_losses = df_losses.rename(columns={
    'Unnamed: 0': 'datetime',
})

In [6]:

df_NTC['datetime'] = pd.to_datetime(df_NTC['datetime'])
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])
df_renewable['datetime'] = pd.to_datetime(df_renewable['datetime'])
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])

In [7]:
df_losses['Wirkverluste/Active Losses'] = pd.to_numeric(df_losses['Wirkverluste/Active Losses'], errors='coerce')

In [8]:
# Convert "Zeitstempel" column to datetime format, if not already
#df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])

# Subtract 15 minutes from each date in the "Zeitstempel" column
df_losses['datetime'] = df_losses['datetime'] - pd.Timedelta(minutes=15)



In [9]:

df_losses_new = df_losses.set_index('datetime')

hourly_loss = df_losses_new.resample('1H', closed='left', label='left').sum()

In [10]:
hourly_loss["Wirkverluste/Active Losses"]=hourly_loss["Wirkverluste/Active Losses"]/1000


In [11]:
#hourly_loss

In [12]:


# Check if the data is already in 6-hour intervals
if (df_temp['datetime'].diff().dt.total_seconds() / 3600).median() == 6:
    # If the data is in 6-hour intervals, leave the datetime column as it is
    hourly_temp = df_temp
else:
    # If the data is not in 6-hour intervals, resample it to hourly intervals
    hourly_temp = df_temp.resample('H', on='datetime').mean().reset_index()

    # Fill missing values in other columns with the values from the initial 6-hourly data
    columns_to_fill = hourly_temp.columns.difference(['datetime'])
    hourly_temp[columns_to_fill] = hourly_temp[columns_to_fill].fillna(method='ffill', limit=6)
    # Replace '6' with the appropriate number if you want to fill more or fewer hours

# Print the resulting hourly data
display(hourly_temp.head(10))

,datetime,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,2019-01-01 01:00:00,4.1067,5.9729,7.4268,4.0281
1,2019-01-01 02:00:00,4.1067,5.9729,7.4268,4.0281
2,2019-01-01 03:00:00,4.1067,5.9729,7.4268,4.0281
3,2019-01-01 04:00:00,4.1067,5.9729,7.4268,4.0281
4,2019-01-01 05:00:00,4.1067,5.9729,7.4268,4.0281
5,2019-01-01 06:00:00,4.1067,5.9729,7.4268,4.0281
6,2019-01-01 07:00:00,1.7595,5.5398,6.3425,2.7281
7,2019-01-01 08:00:00,1.7595,5.5398,6.3425,2.7281
8,2019-01-01 09:00:00,1.7595,5.5398,6.3425,2.7281
9,2019-01-01 10:00:00,1.7595,5.5398,6.3425,2.7281


In [13]:
hourly_loss = hourly_loss.reset_index()

In [14]:
merged_df = df_NTC.merge(hourly_loss, on='datetime', how='outer')
merged_df = merged_df.merge(df_renewable, on='datetime', how='outer')
merged_df = merged_df.merge(hourly_temp, on='datetime', how='outer')

In [15]:
na_counts_per_column = merged_df.isna().sum()

In [16]:
na_counts_per_column 

datetime                         0
CH_AT                            3
CH_DE                            3
CH_FR                            3
CH_IT                            3
AT_CH                            3
DE_CH                            3
FR_CH                            3
IT_CH                            3
Wirkverluste/Active Losses       0
solar_fore_de [MW]               3
solar_fore_it [MW]               3
wind_fore_de [MW]                3
wind_fore_it [MW]                3
temperature_fore_ch           1578
temperature_fore_fr           1578
temperature_fore_de           1578
temperature_fore_it           1578
dtype: int64

In [17]:
merged_df_incomplete = merged_df.dropna()

In [18]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'Wirkverluste/Active Losses': 'Loss'})

In [19]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_de [MW]': 'solar_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_it [MW]': 'solar_fore_it'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_de [MW]': 'wind_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_it [MW]': 'wind_fore_it'})
#drop the datetime

#merged_df_incomplete=merged_df_incomplete.drop('datetime')


In [20]:
merged_df_incomplete = merged_df_incomplete.reset_index()

In [21]:
#merged_df_incomplete['weekday'] = merged_df_incomplete['datetime'].dt.day_name()
#merged_df_incomplete['month'] = merged_df_incomplete['datetime'].dt.month_name()
#merged_df_incomplete['year'] = merged_df_incomplete['datetime'].dt.year

In [22]:
merged_df_incomplete['weekday'] = merged_df_incomplete['datetime'].dt.day_name().astype('category').cat.codes
merged_df_incomplete['month'] = merged_df_incomplete['datetime'].dt.month_name().astype('category').cat.codes
merged_df_incomplete['year'] = merged_df_incomplete['datetime'].dt.year


In [23]:
merged_df_incomplete['year']



0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
24727    2021
24728    2021
24729    2021
24730    2021
24731    2021
Name: year, Length: 24732, dtype: int32

# 2. Create 24 different dataframes

### 2.1. Create 1 Big Dataset with all the Lags

In [24]:
merged_df_incomplete['hour'] = merged_df_incomplete['datetime'].dt.hour

In [25]:
merged_ready_1= merged_df_incomplete

In [26]:
features_many = ['CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
               'solar_fore_de', 'solar_fore_it', 'wind_fore_de', 'wind_fore_it',
               'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
               'temperature_fore_it']

    # Create the lagged features
for feature in features_many:
    for lag in range(1, 49):
        col_name = f"{feature}_lag{lag}"
        merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)

#merged_ready_1 = merged_ready_1.iloc[167:, :]

/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3333389843.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

In [27]:
features_loss = ['Loss']

    # Create the lagged features
for feature in features_loss:
    for lag in range(1, 673):
        col_name = f"{feature}_lag{lag}"
        merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)

merged_ready_1 = merged_ready_1.iloc[673:, :]

/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_181741/3408782732.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

In [28]:
rename_dict = {
    'CH_AT': 'CH_AT_lag0',
    'CH_DE': 'CH_DE_lag0',
    'CH_FR': 'CH_FR_lag0',
    'CH_IT': 'CH_IT_lag0',
    'AT_CH': 'AT_CH_lag0',
    'DE_CH': 'DE_CH_lag0',
    'FR_CH': 'FR_CH_lag0',
    'IT_CH': 'IT_CH_lag0',
    #'Loss': 'Loss_lag0',
    'solar_fore_de': 'solar_fore_de_lag0',
    'solar_fore_it': 'solar_fore_it_lag0',
    'wind_fore_de': 'wind_fore_de_lag0',
    'wind_fore_it': 'wind_fore_it_lag0',
    'temperature_fore_ch': 'temperature_fore_ch_lag0',
    'temperature_fore_fr': 'temperature_fore_fr_lag0',
    'temperature_fore_de': 'temperature_fore_de_lag0',
    'temperature_fore_it': 'temperature_fore_it_lag0'#,
    #'hour': 'hour_lag0'
}

# Rename the columns
merged_ready_1 = merged_ready_1.rename(columns=rename_dict)

In [29]:
num_cols_per_line = 5
for i in range(0, len(merged_ready_1.columns), num_cols_per_line):
    print(', '.join(merged_ready_1.columns[i:i+num_cols_per_line]))

index, datetime, CH_AT_lag0, CH_DE_lag0, CH_FR_lag0
CH_IT_lag0, AT_CH_lag0, DE_CH_lag0, FR_CH_lag0, IT_CH_lag0
Loss, solar_fore_de_lag0, solar_fore_it_lag0, wind_fore_de_lag0, wind_fore_it_lag0
temperature_fore_ch_lag0, temperature_fore_fr_lag0, temperature_fore_de_lag0, temperature_fore_it_lag0, weekday
month, year, hour, CH_DE_lag1, CH_DE_lag2
CH_DE_lag3, CH_DE_lag4, CH_DE_lag5, CH_DE_lag6, CH_DE_lag7
CH_DE_lag8, CH_DE_lag9, CH_DE_lag10, CH_DE_lag11, CH_DE_lag12
CH_DE_lag13, CH_DE_lag14, CH_DE_lag15, CH_DE_lag16, CH_DE_lag17
CH_DE_lag18, CH_DE_lag19, CH_DE_lag20, CH_DE_lag21, CH_DE_lag22
CH_DE_lag23, CH_DE_lag24, CH_DE_lag25, CH_DE_lag26, CH_DE_lag27
CH_DE_lag28, CH_DE_lag29, CH_DE_lag30, CH_DE_lag31, CH_DE_lag32
CH_DE_lag33, CH_DE_lag34, CH_DE_lag35, CH_DE_lag36, CH_DE_lag37
CH_DE_lag38, CH_DE_lag39, CH_DE_lag40, CH_DE_lag41, CH_DE_lag42
CH_DE_lag43, CH_DE_lag44, CH_DE_lag45, CH_DE_lag46, CH_DE_lag47
CH_DE_lag48, CH_FR_lag1, CH_FR_lag2, CH_FR_lag3, CH_FR_lag4
CH_FR_lag5, CH_FR_lag6,

In [30]:
#make sure all the losses with lag are named -1


def modify_col_name(col_name):
    match = re.match(r"Loss_lag(\d+)", col_name)
    if match:
        number = int(match.group(1))
        return f"Loss_lag{number - 1}"
    return col_name

# Apply the function to each column name
merged_ready_1.columns = [modify_col_name(col) for col in merged_ready_1.columns]


In [31]:
# replac 0 with 24 because forecast 1-24
merged_ready_1.loc[merged_ready_1["hour"] == 0, "hour"] = 24



## 2.2 Create 24 datasets

In [32]:
dfs_total = {}

dataset_lags = [673]

for i in dataset_lags:
    working_df = merged_ready_1.copy()
    
    threshold = i-1

    # Extract columns to drop based on their name
    cols_to_drop = []
    for col in working_df.columns:
        match = re.search(r'lag(\d+)', col)
        if match:
            lag_number = int(match.group(1))
            if lag_number > threshold:
                cols_to_drop.append(col)

    # Drop these columns
    working_df = working_df.drop(columns=cols_to_drop)
    
    
    
    dfs_total[f'df_{i}'] = working_df



In [33]:
for key, value in dfs_total.items():
    print(f"Key: {key}, Type: {type(value)}")


Key: df_673, Type: <class 'pandas.core.frame.DataFrame'>


In [34]:
#  delete all the columns with lag lower than threshold

def set_midnight(df_input, threshold):
   
    
    working_df = df_input.copy()

    
    cols_to_drop = []
    for col in working_df.columns:
        match = re.search(r'lag(\d+)', col)
        if match:
            lag_number = int(match.group(1))
            if lag_number < threshold:
                cols_to_drop.append(col)
    working_df = working_df.drop(columns=cols_to_drop)
    
    return working_df

# Example usage:
# result_df = drop_lagged_columns(merged_ready_1, threshold_value)

In [35]:
filtered_datasets = {}
for lag in dataset_lags:
    key = f"df_{lag}"
    df_current = dfs_total[key]
    filtered_datasets[key] = {}
    
    for i in range(1, 25):
        filtered_df = df_current[df_current['hour'] == i]
        filtered_df_filtered=set_midnight(filtered_df, i)
        filtered_datasets[key][i] = filtered_df_filtered



In [36]:
#filtered_datasets["df_72"][3].head(2)

# 3. Run the model to get MAE

### 3.1 Define Model Formula

In [150]:


def compute_mae(merged_df_ready_prep):
    # Ensure the merged_ready DataFrame is created first
    merged_ready = merged_df_ready_prep.copy()
    merged_ready=merged_ready.drop('datetime', axis=1)
   
    X = merged_ready.drop('Loss', axis=1)  # Features
    y = merged_ready['Loss']  # Target variable

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'device': 'gpu',  # Enable GPU support
        
    }

    # Create the LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Train the model
    model = lgb.train(params, train_data)

    # Predict the test data
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    return mae



### 3.2 Run the models

In [151]:




# Lists to store the main keys, sub-keys and the number of rows
main_keys = []
sub_keys = []
num_rows = []
for lag in [48]:
#for lag in dataset_lags:
    dataset_key = "df_" + str(lag)
    if dataset_key in filtered_datasets:
        #for sub_key in range(1, 25):  # since you mentioned it goes from 1 to 24
        for sub_key in range(1, 25):
            main_keys.append(dataset_key)
            sub_keys.append(sub_key)
            num_rows.append(compute_mae(filtered_datasets[dataset_key][sub_key]))

# Create a DataFrame
result_df = pd.DataFrame({
    'Dataset_Key': main_keys,
    'Sub_Key': sub_keys,
    'Num_Rows': num_rows
})

print(result_df)



Empty DataFrame
Columns: [Dataset_Key, Sub_Key, Num_Rows]
Index: []


### 3.3 Save the model

In [ ]:
filename = "results_lags_[" + "_".join(map(str, dataset_lags)) + "].csv"
#dataset_lags_custom=[48]
#filename = "results_lags_[" + "_".join(map(str, dataset_lags_custom)) + "].csv"

In [ ]:
filename

In [ ]:
result_df.to_csv(filename, index=False)

# 4. Run the model and save it

In [37]:

def save_model(merged_df_ready_prep, filename):
    # Check if the folder exists; if not, create it
    folder_path = 'Saved_model_raw'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Create the complete path for the model file
    full_path = os.path.join(folder_path, filename)
    
    merged_ready = merged_df_ready_prep.copy()
    merged_ready = merged_ready.drop('datetime', axis=1)
   
    X = merged_ready.drop('Loss', axis=1)  # Features
    y = merged_ready['Loss']  # Target variable

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'device': 'gpu',  # Enable GPU support
    }

    # Create the LightGBM dataset
    #train_data = lgb.Dataset(X_train, label=y_train)
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=['month','weekday','year'])
    
    # Train the model
    model = lgb.train(params, train_data)

    # Save the trained model
    with open(full_path, 'wb') as file:
        pickle.dump(model, file)

# Example usage
# save_model(your_dataframe, 'model_filename.pkl')


In [38]:




# Lists to store the main keys, sub-keys and the number of rows

sub_keys = []

for lag in dataset_lags:
#for lag in dataset_lags:
    dataset_key = "df_" + str(lag)
    if dataset_key in filtered_datasets:
        #for sub_key in range(1, 25):  # since you mentioned it goes from 1 to 24
        for sub_key in range(1, 25):
            filename_input=str(dataset_key)+"_hour_"+str(sub_key)
            save_model(filtered_datasets[dataset_key][sub_key],filename_input)

# Create a DataFrame
result_df = pd.DataFrame({
    'Dataset_Key': main_keys,
    'Sub_Key': sub_keys,
    'Num_Rows': num_rows
})

print(result_df)



[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 279323
[LightGBM] [Info] Number of data points in the train set: 991, number of used features: 1369
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1353 dense feature groups (1.28 MB) transferred to GPU in 0.098114 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 105.444590
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 277187
[LightGBM] [Info] Number of data points in the train set: 99

KeyboardInterrupt: 